In [ ]:
!pip install beautifulsoup4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from bs4 import BeautifulSoup

In [ ]:
import requests as rq
import pandas as pd

In [ ]:
matchResultsUrl = "https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?id=14450;type=tournament"
response = rq.get(matchResultsUrl)

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
rows = soup.find('table', {'class':'engineTable'}).find('tbody').find_all('tr')

In [ ]:
#retrieve general match data including links to individual match details
matchData = []
matchLinks = []

dataHeaders = ["team1",	"team2", "winner","margin", "ground",	"matchDate",	"match_id"]

for row in rows:
  rowDict = {}
  for i in range(0, 7):
    rowDict[dataHeaders[i]] = row.find_all('td')[i].text

  matchData.append(rowDict)
  if(rowDict["winner"]=="abandoned" or rowDict["winner"]=="no result"):
    samURL = "http://stats.espncricinfo.com" + row.find_all("a")[3]['href']
  else:
    samURL = "http://stats.espncricinfo.com" + row.find_all("a")[4]['href']
  matchLinks.append(samURL)

df_matchSummary = pd.DataFrame(matchData)

In [ ]:
#save match links for additional data scraping
import csv

#save the matchSummary df as csv file
df_matchSummary.to_csv('dim_match_summary.csv')

with open('matchLinks.csv', 'w') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerows(rows)

#player data scrapped using bright data web scrapping tool